# Hacking the Python API for Caffe

Often, when doing some advanced experiments with Caffe, a function or two are missing in the python API. This tutorial will cover typical ways in which you can add a function, or parameters, to the pycaffe API.

This tutorial will not have many runnable code examples, because it relies on a modification of the caffe code, which makes it unpractical.

## Caffe organization

The repository is organized as such:

- `src`
  - `caffe`: C++/CUDA implementation of caffe
    - `layers`: layer definitions
    - `solvers`: different solver implementations
    - `test`: C++ unit tests
- `include`
  - `caffe`: C++ headers
    - `layers`: layer declarations
    - `test`: headers for the tests
- `python`
  - misc command-line tools
  - `caffe`: Caffe module implementation
    - `__init__.py`: module code, run on import, for declarations
    - `_caffe.cpp`: Python/C++ bindings, with Boost::Python
    - `io.py`: various input/output features
    - `net_spec.py`: implementation of the net specification object
    - `pycaffe.py`: Python-specific Caffe code
    - `test`: Python unit tests
    

## Adding a pure python function

### To a module

Adding a function to a submodule (e.g. `caffe.io`) is very simple, just add the function in the `python/caffe/io.py` file.

Adding it to the `caffe` module is slightly more involved: you have to modify `python/caffe/__init__.py` to make sure that it is imported into the scope of the module.

### To a class

To add a python-implemented function to a class, add it in `pycaffe.py`. Thanks to python's very lax typing system, you can just write `<class>.<function> = my_function`. You have the example of `Net`:

```python
def _Net_forward(self, blobs=None, start=None, end=None, **kwargs):
    ...
    
Net.forward = _Net_forward
```

This adds the possibility of running `net.forward()` with a caffe net. Note that the first argument of `_Net_forward` is `self`, the net the function is called on.

To make it a property (field) rather than a function (e.g. `blobs`), your function must take only one argument, `self`, and either be annotated with `@property`, or assigned with the property function (e.g. `top_names`):

```python
# This function will be called when accessing net.blobs
@property
def _Net_blobs(self):
    ...

class _Net_IdNameWrapper:
    """
    A simple wrapper that allows the ids propery to be accessed as a dict
    indexed by names. Used for top and bottom names
    """
    def __init__(self, net, func):
        ...

# Set the field `blobs` to call _Net_blobs
Net.blobs = _Net_blobs
# Set the field `top_names` to call that lambda, and get an instance of _NetIdNameWrapper
Net.top_names = property(lambda n: _Net_IdNameWrapper(n, Net._top_ids))
```

If the class you want to modify is not in pycaffe, simply import it at the top from `_caffe`, and adapt `__init__.py` to import it from the right place

## Adding a binding to a C++ function

Often, simply adding a python wrapper is not enough, and you will need to call directly a C++ function:

- To access implementation details, such as private fields
- For better performance

In order to do that, you will need to add the binding to `python/caffe/_caffe.cpp`. At the end of the file, all the classes are defined, with their fields and methods bound to C++ fields/methods.

A simple field declaration will look like this (in the Blob definition):

```C++
.add_property("num", &Blob<Dtype>::num)
```

This will bind the field `num` of the python `Blob` class to call the `num()` function of the C++ `Blob<Dtype>` class.

The same syntax with `.def` instead of `.add_property` will yield a method instead of a field. You can implement the function in C++ and bind it directly (see `net.save`):

```C++
# Declare the function
void Net_Save(const Net<Dtype>& net, string filename) {
    ...
}

...

bp::class_<Net<Dtype>>("Net", bp::no_init)
# Now we can call net.save(file)
.def("save", &Net_Save)
```

More complex matters (overloaded functions, return by reference, memory management, ...) require a bit of knowledge of Boost::Python (http://www.boost.org/doc/libs/1_58_0/libs/python/doc/tutorial/doc/html/index.html). Usually, looking at the examples around will provide the needed documentation.

**Important:** As you changed the C++ code, you will have to recompile before running your code (`make pycaffe`).

## Contributing

Now that you have improved the `pycaffe` interface, it's time to give back to the community! You are using open source code, take some time to contribute!

The first step is to clean up your code, trim what's unnecessary, make sure you are under the 80 columns limit, and that you are not using anything specific to either Python 2 or 3.

Next, you will want to add some tests to make sure that your code behaves correctly, and that if someone breaks it, it will show. If you changed some C++ code, add a test in `src/caffe/test`, and for Python it's in `python/caffe/test`.

Once you have that, make sure you are on top of the current `master` branch of the repository, clean up your commits (`git rebase --interactive master` can help you), and push to your repository/branch. Then simply create a pull request on github, and you'll be set!